In [1]:
import torch
from torchvision.datasets import VOCDetection
from pathlib import Path
import time
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from IPython.display import clear_output
from torchinfo import summary
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import random_split
import json
import matplotlib.pyplot as plt

import sys
sys.path.append("../src")

from dataset import ObjectDetectionDataset
from utils import plot_datapoint

KeyboardInterrupt: 

In [2]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [3]:
train_dataset = VOCDetection(data_path, year="2007", image_set="train", download=True)

Using downloaded and verified file: ..\data\VOCtrainval_06-Nov-2007.tar
Extracting ..\data\VOCtrainval_06-Nov-2007.tar to ..\data


In [5]:
train_dataset[0]


(<PIL.Image.Image image mode=RGB size=500x333>,
 {'annotation': {'folder': 'VOC2007',
   'filename': '000012.jpg',
   'source': {'database': 'The VOC2007 Database',
    'annotation': 'PASCAL VOC2007',
    'image': 'flickr',
    'flickrid': '207539885'},
   'owner': {'flickrid': 'KevBow', 'name': '?'},
   'size': {'width': '500', 'height': '333', 'depth': '3'},
   'segmented': '0',
   'object': [{'name': 'car',
     'pose': 'Rear',
     'truncated': '0',
     'difficult': '0',
     'bndbox': {'xmin': '156', 'ymin': '97', 'xmax': '351', 'ymax': '270'}}]}})

In [29]:
classes = [
    "person",
    "bird", "cat", "cow", "dog", "horse", "sheep",
    "aeroplane", "bicycle", "boat", "bus", "car", "motorbike", "train",
    "bottle", "chair", "diningtable", "pottedplant", "sofa", "tvmonitor"
]

# Create the class_to_idx dictionary
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

print(class_to_idx)


{'person': 0, 'bird': 1, 'cat': 2, 'cow': 3, 'dog': 4, 'horse': 5, 'sheep': 6, 'aeroplane': 7, 'bicycle': 8, 'boat': 9, 'bus': 10, 'car': 11, 'motorbike': 12, 'train': 13, 'bottle': 14, 'chair': 15, 'diningtable': 16, 'pottedplant': 17, 'sofa': 18, 'tvmonitor': 19}


In [7]:
dataset = ObjectDetectionDataset(data_path/"single_object_images.pt")
image, target = dataset[0]

c:\Users\sampath\Dev\ObjectDetection\notebooks\../src\dataset.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.single_object_images = torch.load(path)


In [31]:
train_dataset, val_dataset = random_split(dataset, [0.7, 0.3])
len(train_dataset), len(val_dataset)        

(634, 271)

In [ ]:
torchvision.models.resnet18()

In [70]:
transforms = ResNet18_Weights.IMAGENET1K_V1.transforms()

In [117]:
preprocessed_image = transforms(image).unsqueeze(dim=0)
logits = model(preprocessed_image).flatten()
probs = F.softmax(logits, dim=-1)
pred = probs.argmax().item()
print(pred)
ResNet18_Weights.IMAGENET1K_V1.meta["categories"][pred]

232


'Border collie'

In [103]:
for name in ResNet18_Weights.IMAGENET1K_V1.meta["categories"]:
    if name.lower() == "vehicle":
        print(name)